In [3]:
!rm -r RMDL

In [4]:
!git clone --recursive https://github.com/kk7nc/RMDL.git

Cloning into 'RMDL'...
remote: Enumerating objects: 2311, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 2311 (delta 71), reused 120 (delta 61), pack-reused 2179
Receiving objects: 100% (2311/2311), 227.86 MiB | 20.89 MiB/s, done.
Resolving deltas: 100% (1536/1536), done.


In [5]:
cd ./RMDL

/content/RMDL


In [6]:
!pip  install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating RMDL.egg-info
writing RMDL.egg-info/PKG-INFO
writing dependency_links to RMDL.egg-info/dependency_links.txt
writing requirements to RMDL.egg-info/requires.txt
writing top-level names to RMDL.egg-info/top_level.txt
writing manifest file 'RMDL.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'RMDL.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/RMDL
copying RMDL/__init__.py -> build/lib/RMDL
copying RMDL/Plot.py -> build/lib/RMDL
copying RMDL/RMDL_Image.py -> build/lib/RMDL
copying RMDL/BuildModel.py -> build/lib/RMDL
copying RMDL/RMDL_Text.py -> build/lib/RMDL
copying RMDL/Global.py -> build/lib/RMDL
copying RMDL/text_feature_extraction.py -> build/lib/RMDL
creating build/lib/RMDL/Download
copying RMDL/Download/__init__.py -> build/lib/RMDL/Download
copying RMDL/Download/Download_WOS.p

In [8]:
!cat ./RMDL/BuildModel.py > ./RMDL/BuildModel_copy.py
!cat ./RMDL/text_feature_extraction.py > ./RMDL/text_feature_extraction_copy.py
!cat ./RMDL/RMDL_Text.py > ./RMDL/RMDL_Text_copy.py

In [9]:
!sed 's/from keras.layers.merge import Concatenate/from keras.layers import Concatenate/g' ./RMDL/BuildModel_copy.py > ./RMDL/BuildModel.py
!cat ./RMDL/BuildModel.py > ./RMDL/BuildModel_copy.py

In [10]:
!sed 's/model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))/model.add(GRU(gru_node,return_sequences=True))/g' ./RMDL/BuildModel_copy.py > ./RMDL/BuildModel.py
!cat ./RMDL/BuildModel.py > ./RMDL/BuildModel_copy.py

In [11]:
!sed 's/model.add(GRU(gru_node, recurrent_dropout=0.2))/model.add(GRU(gru_node))/g' ./RMDL/BuildModel_copy.py > ./RMDL/BuildModel.py

In [12]:
!sed 's/from keras.preprocessing.sequence import pad_sequences/from keras_preprocessing.sequence import pad_sequences/g' ./RMDL/text_feature_extraction_copy.py > ./RMDL/text_feature_extraction.py
!cat ./RMDL/text_feature_extraction.py > ./RMDL/text_feature_extraction_copy.py

In [13]:
!sed 's/text = np.concatenate((X_train, X_test), axis=0)/text = X_train + X_test/g' ./RMDL/text_feature_extraction_copy.py > ./RMDL/text_feature_extraction.py
!cat ./RMDL/text_feature_extraction.py > ./RMDL/text_feature_extraction_copy.py

In [14]:
!sed 's/text = np.array(text)//g' ./RMDL/text_feature_extraction_copy.py > ./RMDL/text_feature_extraction.py

In [15]:
!sed 's/                                          batch_size=batch_size)//g' ./RMDL/RMDL_Text_copy.py > ./RMDL/RMDL_Text.py
!cat ./RMDL/RMDL_Text.py > ./RMDL/RMDL_Text_copy.py

In [16]:
!sed 's/y_pr_ = model_tmp.predict_classes(x_test_tfidf,/y_pr_ = np.argmax(model_tmp.predict(x_test_tfidf), axis=-1)/g' ./RMDL/RMDL_Text_copy.py > ./RMDL/RMDL_Text.py
!cat ./RMDL/RMDL_Text.py > ./RMDL/RMDL_Text_copy.py

In [17]:
!sed 's/y_pr_ = model_tmp.predict(x_test_tfidf,/y_pr_ = model_tmp.predict(x_test_tfidf,batch_size=batch_size)/g' ./RMDL/RMDL_Text_copy.py > ./RMDL/RMDL_Text.py
!cat ./RMDL/RMDL_Text.py > ./RMDL/RMDL_Text_copy.py

In [18]:
!sed 's/y_pr_ = model_tmp.predict_classes(x_test_embedded, batch_size=batch_size)/y_pr_ = np.argmax(model_tmp.predict(x_test_embedded, batch_size=batch_size), axis=1)/g' ./RMDL/RMDL_Text_copy.py > ./RMDL/RMDL_Text.py

In [19]:
!rm ./RMDL/BuildModel_copy.py
!rm ./RMDL/text_feature_extraction_copy.py
!rm ./RMDL/RMDL_Text_copy.py

# IMPORT, CONFIG

In [46]:
from RMDL import RMDL_Text as RMDL

import os
import re
import random

import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [21]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [22]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [23]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# LOAD DATA

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd /content/drive/MyDrive/Professional11

/content/drive/MyDrive/Professional11


In [26]:
train_data = pd.read_csv('./train.csv')

In [27]:
train_data.head(20)

,id,review_text,thumbs_up,score
0,0,It provides latest informations. Easy to use. ...,0,5
1,1,Good service.,0,4
2,2,Looks great and simple to use.,0,4
3,3,"Clear stats, everthing you need to know at a g...",0,4
4,4,Very good app,0,5
5,5,this helps me keep track of all my balances in...,0,5
6,6,I love how it tells information about the comp...,0,5
7,7,Very exact! An eye opener,0,4
8,8,"Never made a budget before, this app made it s...",0,5
9,9,Would love it but I can't put my SunTrust Mort...,0,4


In [59]:
test_data = pd.read_csv('test.csv')['review_text'].iloc[::10]

In [60]:
test_data

0        I didnt expect it to be real. But just played ...
10               I like seeing my watch list and articles.
20       Loved the in-depth analysis of every stock pro...
30       Works great, exactly what I want in a stock tr...
40       This app is Great!. It updates with the latest...
                               ...                        
31860                                        Very useful 👍
31870                        One of the best apps EVER !!!
31880    The Camry of budgeting apps. Not fancy, but ge...
31890                      Stoped updating my bank account
31900                    Good but is it only for usa alone
Name: review_text, Length: 3191, dtype: object

In [30]:
target_column = 'score'
data = train_data[['review_text',target_column]]

In [31]:
data

,review_text,score
0,It provides latest informations. Easy to use. ...,5
1,Good service.,4
2,Looks great and simple to use.,4
3,"Clear stats, everthing you need to know at a g...",4
4,Very good app,5
...,...,...
127628,5yr/ longer timeframe charts please.,2
127629,My morning app.,5
127630,Changes in the portfolio isn't working,2
127631,"Literal scum company, screwing over the little...",1


In [32]:
X = data.review_text.to_list()

In [33]:
Y = data[target_column].to_numpy()

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=RANDOM_SEED,stratify=data[target_column].to_list())

In [35]:
len(y_train), len(y_test)

(85514, 42119)

# TRAIN

MAX_NB_WORDS=20000 -- x_train_tfidf ~ 4.7 Gb, x_test_tfidf ~ 2.3 Gb

In [36]:
batch_size = 450
sparse_categorical = 0

In [37]:
n_epochs = [1, 1, 2]  ## DNN--RNN-CNN
Random_Deep = [0,0,1]  ## DNN--RNN-CNN

In [ ]:
RMDL.Text_Classification(X_train, y_train, X_test, y_test,
                         batch_size=batch_size,
                         sparse_categorical=True,
                         random_deep=Random_Deep,
                         epochs=n_epochs,
                         MAX_NB_WORDS=12500,
                         GloVe_dir='Glove',
                         random_state=RANDOM_SEED)

Done1
Glove/glove.6B.50d.txt
Found 32359 unique tokens.
(127633, 500)
Total 400001 word vectors.
6
CNN 0
Filter   8
Node   185
Epoch 1/2
Error in model 0 try to re-generate an other model
CNN 0
Filter   8
Node   267
Epoch 1/2


In [38]:
model_end_cnn = tf.keras.models.load_model('weights\weights_CNN_0.hdf5')

In [39]:
model_end_cnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 500, 50)      3293100     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 499, 505)     51005       ['embedding_1[0][0]']            
                                                                                                  
 conv1d_1 (Conv1D)              (None, 498, 505)     76255       ['embedding_1[0][0]']            
                                                                                              

In [62]:
def loadData_Tokenizer(X_train, X_test,GloVe_DIR,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH,EMBEDDING_DIM):
    print('begin tokenize')
    np.random.seed(7)
    text = X_train #text = np.concatenate((X_train, X_test), axis=0)
    #text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open(GloVe_DIR, encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    print('end tokenize')
    return (X_train, X_test, word_index,embeddings_index)

In [63]:
GloVe_DIR = './Glove/glove.6B.50d.txt'

In [64]:
MAX_NB_WORDS = 20000

In [65]:
MAX_SEQUENCE_LENGTH = 500

In [66]:
EMBEDDING_DIM = 50

In [67]:
test_data_embedded, x_test_embedded, word_index, embeddings_index = loadData_Tokenizer(test_data, [],GloVe_DIR,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH,EMBEDDING_DIM)

begin tokenize
Found 4599 unique tokens.
(3191, 500)
Total 400001 word vectors.
end tokenize


In [68]:
test_data_embedded.shape

(3191, 500)

In [75]:
if sparse_categorical:
    model_end_cnn.compile(loss='sparse_categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])
else:
    model_end_cnn.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

y_pr_ = model_end_cnn.predict(test_data_embedded, batch_size=batch_size)
y_pr_ = np.argmax(y_pr_, axis=1)

8/8 [==============================] - 84s 10s/step


In [76]:
len(y_pr_)

3191